<a href="https://www.kaggle.com/code/ocanaydin/shoe-bot-sandal-pytorch-tfl?scriptVersionId=113973037" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LIBRARIES**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

**DATA TRANSFORMS**

In [ ]:
"""Data Augmentation and Normalization for train images.Test and Val images are proccessed only with normalization"""
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.76891514,0.77947596,0.80775537],[0.33321657,0.32545126,0.30419493])
    ])

**LOAD IMAGES**

In [ ]:
data_dir = "../input/shoe-vs-sandal-vs-boot-dataset-15k-images/Shoe vs Sandal vs Boot Dataset"
dataset = datasets.ImageFolder(data_dir,transform = data_transforms)
"""Split dataset as train,test and val."""
train_size = int(len(dataset) * 0.7)
val_size = (len(dataset) - train_size) // 2
test_size = val_size
print(f"Train Size : {train_size} Val Size : {val_size} Test Size : {test_size}")

**DIVIDE DATASET**

In [ ]:
"""Use random split to divide dataset."""
train_data,val_data,test_data = torch.utils.data.random_split(dataset,[train_size,val_size,test_size],
                                                              generator = torch.Generator().manual_seed(42))

class_names = train_data.dataset.classes


**DATA LOADERS**

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data,batch_size = batch_size,shuffle = True)
val_loader = torch.utils.data.DataLoader(val_data,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size = batch_size,shuffle = True)

**VISUALIZATION OF IMAGES**

In [ ]:
import matplotlib.pyplot as plt
def imshow(inp,title = None):
    inp = inp.numpy().transpose((1,2,0))
    mean = np.array([0.76891514,0.77947596,0.80775537])
    std = np.array([0.33321657,0.32545126,0.30419493])
    inp = std * inp + mean
    inp = np.clip(inp,0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

"""Get a batch of training_data"""
inputs,classes = next(iter(test_loader))
"""Make a grid from batch."""
out = torchvision.utils.make_grid(inputs[0 : 6])
imshow(out,title = [class_names[x] for x in classes[0:6]])

**TRAINING**

In [ ]:
dataloaders = {"train" : train_loader,"val" : val_loader,"test" : test_loader}
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
dataset_sizes = {"train" : train_size,"val" : val_size,"test" : test_size}
def train_model(model,criterion,optimizer,scheduler,num_epochs = 25):
   #Define time and best model weights to save.
   since = time.time()
   best_model_wts = copy.deepcopy(model.state_dict())
   best_acc = 0.0
   for epoch in range(num_epochs):
     print(f"Epoch : {epoch+1} / {num_epochs }")
     print("-" * 10)
     #Each epoch has training and validation phase.
     for phase in ["train","val"]:
       if phase == "train":
         model.train()
       else:
         model.eval()
       running_loss = 0.0
       running_corrects = 0
       #Iterate over data.
       for inputs,labels in dataloaders[phase]:
         #Move tensors to gpu.
         inputs = inputs.to(device)
         labels = labels.to(device)
         #Zero Gradients.
         optimizer.zero_grad()
         #Forward
         #Track history if only in train.
         with torch.set_grad_enabled(phase == "train"):
           #Get preds in train and calculate loss.
           outputs = model(inputs)
           _,preds = torch.max(outputs,1)
           loss = criterion(outputs,labels)
           #Backward in train phase.
           if phase == "train":
             loss.backward()
             optimizer.step()
         #statistics
         running_loss += loss.item() * inputs.size(0)
         running_corrects += torch.sum(preds == labels.data)
    
       if phase == "train":
         scheduler.step()

       epoch_loss = running_loss / dataset_sizes[phase]
       epoch_acc = running_corrects.double() / dataset_sizes[phase]

       print(f"{phase} Loss : {epoch_loss} Acc {epoch_acc}")
       #Deep copy of the model.
       if phase == "val" and epoch_acc > best_acc:
         best_acc = epoch_acc
         best_model_wts = copy.deepcopy(model.state_dict())

     print()
  
   time_elapsed = time.time() - since
   print(f"Training completed in : {time_elapsed // 60}m in {time_elapsed % 60}")
   print(f"Best val acc : {best_acc}")

   #Load model best weights.
   model.load_state_dict(best_model_wts)
   return model



**FINETUNING THE CONVNET**

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_features = model_ft.fc.in_features
#Output layer
model_ft.fc = nn.Linear(in_features=num_features,out_features = len(class_names))
#Move to gpu and define loss.
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
#Optimizer
optimizer_ft = optim.SGD(model_ft.parameters(),lr = 0.001,momentum = 0.9)
#Decay LR by a factor 0.1
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,step_size = 5,gamma = 0.1)

In [ ]:
model = train_model(model_ft,criterion,optimizer_ft,exp_lr_scheduler,3)

In [ ]:
def visualize_model(model,num_images = 6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    
    with torch.no_grad():
        for i,(inputs,labels) in enumerate(dataloaders["val"]):
            #Move tensors to gpu
            inputs = inputs.to(device)
            labels = labels.to(device)
            #Forward
            outputs = model(inputs)
            _,preds = torch.max(outputs,1)
            for j in range(inputs.size()[0]):
                #Show validation images and predictions.
                images_so_far += 1
                ax = plt.subplot(num_images // 2,2,images_so_far)
                ax.axis("off")
                ax.set_title(f"Predicted : {class_names[preds[j]]}")
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode = was_training)
                    return
    model.training(mode = was_training)

In [ ]:
visualize_model(model_ft,6)
plt.show()

**SAVE MODEL**

In [ ]:
import os
torch.save(model.state_dict(),"shoe_bot_sandal_TFL.h5")

**LOAD MODEL**

In [ ]:
model1 = model_ft
model1.load_state_dict(torch.load("shoe_bot_sandal_TFL.h5"))

**GET IMAGE FROM INTERNET AND PREDICT IT**

In [ ]:
from PIL import Image
import requests
from io import BytesIO
def get_process_img(url):
    #Load image from url.
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    #Apply transform to image as train,test images.
    img = data_transforms(img)
    return img
    

In [ ]:
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
model = model.to(device)
def pred_single_img(img):
    #Move img to device.
    img = img.to(device)
    #Prediction.
    outputs = model(img.unsqueeze(0))
    _,preds = torch.max(outputs,1)
    #Return prediction with using indexes.
    result = class_names[preds.item()]
    return result

In [ ]:
"""Paste url and get img."""
url = "https://productimages.hepsiburada.net/s/58/600-800/11315175456818.jpg"
img = get_process_img(url)
"""Print pred."""
pred = pred_single_img(img)
print(f"Prediction is : {pred}")
"""Show img."""
plt.imshow(img.permute(1,2,0))
plt.show()